In [3]:
!pip install flask-ngrok

In [4]:
!pip install surprise

     |████████████████████████████████| 11.8MB 4.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670955 sha256=0d62d36f6d94303101af10515d972b6d1fcac2af42193fc778ad8eca8f2f6b4f
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [5]:
import pandas as pd
import pickle
import requests
import json
from flask import Flask, request, jsonify
import surprise
from flask_ngrok import run_with_ngrok
from flask import Flask
import re

In [6]:
def string_preprocess(row):
  text = row
  text = str(text)
  text = re.sub(r'\(\d+\)', '', text)
  text = text.lower()
  return text

In [22]:
app = Flask(__name__)
run_with_ngrok(app)
movie_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/movie.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/rating.csv')
link_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/link.csv')
combined_small = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/combined_small_v2.csv')
model = surprise.dump.load('/content/drive/MyDrive/Colab Notebooks/movie_lens/my_model_v3')
predictions = model[0]
print(predictions[0])
#create index item name and value item id series
movie_rating_combined = pd.merge(movie_df,rating_df,on='movieId')
movie_info_small = movie_rating_combined.iloc[:5000000]
movie_info_small['title'] = movie_info_small['title'].apply(string_preprocess)
movies_list_id = pd.Series(movie_info_small['movieId'].unique(),
                            index=movie_info_small['title'].unique())


@app.route('/api',methods=['GET'])
def get_api_recommendation():
  item_id = []
  movie_name = request.args.get('title')
  movie_id = movies_list_id.loc[movies_list_id.index == movie_name][0]
  user_id = combined_small.loc[combined_small.item == movie_id]['user'].tolist()[:5]
  for id in user_id:
    for i in range(len(predictions)):
      if predictions[i].uid == id:
        item_id.append(predictions[i].iid)
  movies_names = movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[:5]
  return jsonify(movies_names)

@app.route('/movie',methods=['GET'])
def get_movies_recommendation():
  movie_name = request.args.get('title')
  movie_name = movie_name.lower()
  item_id = []
  rating = []
  movie_id = movies_list_id.loc[movies_list_id.index.str.contains(movie_name)][0]
  user_id = combined_small.loc[combined_small.item == movie_id]['user'].tolist()[:5]
  for id in user_id:
    for i in range(len(predictions)):
      if predictions[i].uid == id:
        item_id.append(predictions[i].iid)
        rating.append(predictions[i].est)
  movies_names = movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[:5]
  for names in movies_names:
    if names.strip() == movie_name:
      movies_names.remove(names)
      movies_names.append(movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[6])
  final = dict(zip(movies_names,rating))
  return jsonify(final)

@app.route('/tmdb',methods=['GET'])
def tmdb_call():
  movie_name = request.args.get('title')
  movie_name = movie_name.lower()
  item_id = []
  rating_id = []
  movie_id = movies_list_id.loc[movies_list_id.index.str.contains(movie_name)][0]
  user_id = combined_small.loc[combined_small.item == movie_id]['user'].tolist()[:5]
  for id in user_id:
    for i in range(len(predictions)):
      if predictions[i].uid == id:
        item_id.append(predictions[i].iid)
        rating_id.append((predictions[i].est,predictions[i].iid))
  movies_names = movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[:5]
  final = dict(zip(movies_names,rating_id))
  movies = final
  movie_ids = []
  datas = []

  #getting the ids of the movies
  for key,value in movies.items():
    movie_ids.append(value[1])

  #collecting the tmdb id for link_df
  tmdb_ids = link_df.loc[link_df.movieId.isin(movie_ids)]['tmdbId'].values.tolist()
  for id in tmdb_ids:
    url = f"https://api.themoviedb.org/3/movie/{id}?api_key=f86248c492db78432f0f03f7246698d6"
    resp = requests.get(url=url)
    data = resp.json()
    datas.append(data)
  
  return jsonify(datas)

if __name__=='__main__':
    app.run()
    

user: 16423      item: 7153       r_ui = 3.50   est = 5.00   {'was_impossible': False}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a5fa7ad1e876.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Nov/2020 19:53:57] "GET /tmdb?title=virtuosity HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 19:54:28] "GET /tmdb?title=toy%20story HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 19:54:44] "GET /tmdb?title=toy%20story HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 19:54:53] "GET /tmdb?title=virtuosity HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 19:55:44] "GET /tmdb?title=jumanji HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 19:56:01] "GET /tmdb?title=american%20president HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 19:56:28] "GET /tmdb?title=to%20die%20for HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 19:58:50] "GET /tmdb?title=breakfast HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 20:08:03] "GET /tmdb?title=toy%20story HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2020 20:08:27] "GET /tmdb?title=time HTTP/1.1" 200 -
[2020-11-27 20:33:50,701] ERROR in app: Exception on /tmdb [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_ap